## ANN-Based Churn Classifier

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.preprocessing import OneHotEncoder

In [4]:
## Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Preprocess the data
### Drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
data.isnull().sum()


CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
## Divide the dataset into independent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
## Onehot encode 'Gender'
onehot_encoder_gender=OneHotEncoder( drop='first')
gender_encoder_train=onehot_encoder_gender.fit_transform(X_train[['Gender']]).toarray()
gender_encoder_train

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

In [9]:
gender_encoded_df_train=pd.DataFrame(gender_encoder_train,columns=onehot_encoder_gender.get_feature_names_out(['Gender']), index=X_train.index)
gender_encoded_df_train

,Gender_Male
9254,1.0
1561,1.0
1670,1.0
6087,0.0
6669,1.0
...,...
5734,1.0
5191,0.0
5390,0.0
860,1.0


In [10]:
## Combine one hot encoder column with the original data
X_train=pd.concat([X_train.drop('Gender',axis=1),gender_encoded_df_train],axis=1)
X_train.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male
9254,686,France,32,6,0.00,2,1,1,179093.26,1.0
1561,632,Germany,42,4,119624.60,2,1,1,195978.86,1.0
1670,559,Spain,24,3,114739.92,1,1,0,85891.02,1.0
6087,561,France,27,9,135637.00,1,1,0,153080.40,0.0
6669,517,France,56,9,142147.32,1,0,0,39488.04,1.0


In [11]:
gender_encoder_test=onehot_encoder_gender.transform(X_test[['Gender']]).toarray()
gender_encoder_test

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]])

In [12]:
gender_encoded_df_test=pd.DataFrame(gender_encoder_test,columns=onehot_encoder_gender.get_feature_names_out(['Gender']), index=X_test.index)
gender_encoded_df_test

,Gender_Male
6252,1.0
4684,1.0
1731,0.0
4742,1.0
4521,0.0
...,...
6412,0.0
8285,1.0
7853,0.0
1095,1.0


In [13]:
## Combine one hot encoder column with the original data
X_test=pd.concat([X_test.drop('Gender',axis=1),gender_encoded_df_test],axis=1)
X_test.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male
6252,596,Germany,32,3,96709.07,2,0,0,41788.37,1.0
4684,623,France,43,1,0.00,2,1,1,146379.30,1.0
1731,601,Spain,44,4,0.00,2,1,0,58561.31,0.0
4742,506,Germany,59,8,119152.10,2,1,1,170679.74,1.0
4521,560,Spain,27,7,124995.98,1,1,1,114669.79,0.0


In [14]:
## Onehot encode 'Geography
onehot_encoder_geo=OneHotEncoder(drop='first')
geo_encoder_train=onehot_encoder_geo.fit_transform(X_train[['Geography']]).toarray()
geo_encoder_train

array([[0., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 0.],
       [0., 0.],
       [1., 0.]])

In [15]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_Germany', 'Geography_Spain'], dtype=object)

In [16]:
geo_encoded_df_train=pd.DataFrame(geo_encoder_train,columns=onehot_encoder_geo.get_feature_names_out(['Geography']), index=X_train.index)
geo_encoded_df_train

,Geography_Germany,Geography_Spain
9254,0.0,0.0
1561,1.0,0.0
1670,0.0,1.0
6087,0.0,0.0
6669,0.0,0.0
...,...,...
5734,0.0,0.0
5191,0.0,0.0
5390,0.0,0.0
860,0.0,0.0


In [17]:
## Combine one hot encoder column with the original data
X_train=pd.concat([X_train.drop('Geography',axis=1),geo_encoded_df_train],axis=1)
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
9254,686,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,42,4,119624.60,2,1,1,195978.86,1.0,1.0,0.0
1670,559,24,3,114739.92,1,1,0,85891.02,1.0,0.0,1.0
6087,561,27,9,135637.00,1,1,0,153080.40,0.0,0.0,0.0
6669,517,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0


In [18]:
geo_encoder_test=onehot_encoder_geo.transform(X_test[['Geography']]).toarray()
geo_encoder_test

array([[1., 0.],
       [0., 0.],
       [0., 1.],
       ...,
       [0., 0.],
       [0., 0.],
       [1., 0.]])

In [19]:
geo_encoded_df_test=pd.DataFrame(geo_encoder_test,columns=onehot_encoder_geo.get_feature_names_out(['Geography']), index=X_test.index)
geo_encoded_df_test

,Geography_Germany,Geography_Spain
6252,1.0,0.0
4684,0.0,0.0
1731,0.0,1.0
4742,1.0,0.0
4521,0.0,1.0
...,...,...
6412,1.0,0.0
8285,0.0,0.0
7853,0.0,0.0
1095,0.0,0.0


In [20]:
## Combine one hot encoder column with the original data
X_test=pd.concat([X_test.drop('Geography',axis=1),geo_encoded_df_test],axis=1)
X_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
6252,596,32,3,96709.07,2,0,0,41788.37,1.0,1.0,0.0
4684,623,43,1,0.00,2,1,1,146379.30,1.0,0.0,0.0
1731,601,44,4,0.00,2,1,0,58561.31,0.0,0.0,1.0
4742,506,59,8,119152.10,2,1,1,170679.74,1.0,1.0,0.0
4521,560,27,7,124995.98,1,1,1,114669.79,0.0,0.0,1.0


In [ ]:
## Save the encoders 
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('onehot_encoder_gender.pkl','wb') as file:
    pickle.dump(onehot_encoder_gender,file)

In [22]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [24]:
X_train

array([[ 0.35649971, -0.6557859 ,  0.34567966, ...,  0.91324755,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.29493847, -0.3483691 , ...,  0.91324755,
         1.72572313, -0.57638802],
       [-0.96147213, -1.41636539, -0.69539349, ...,  0.91324755,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ..., -1.09499335,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.3900109 ,  1.03972843, ...,  0.91324755,
        -0.57946723, -0.57638802],
       [ 0.47065475,  1.15059039, -1.38944225, ...,  0.91324755,
         1.72572313, -0.57638802]])

In [ ]:
#save the scaler model
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)